# Differential (gene) Expression

[intro...]

differential_expression()

Description
- This module performs differential expression analysis between two or more phenotypes. For example, you could use this module to find the most differentially expressed genes between a groups of healthy and cancerous cell lines. 
Parameters
- input_gene_expression
  + Gene expression data filename (.gct or .txt file) or python DataFrame object where rows are genes and columns are samples.
- phenotypes_labels
  + Phenotype of each sample in input_gene_expression file (array, Series where indices match , or row of GCT file)
- n_top_hits_to_display
  + The number of most differentially expressed genes to show in the output plot (e.g., if this value is 5, a total of 10 genes (up/down-regulated) will be shown for each phenotype comparison).
  + default = 20
- ranking_method 
  + The function used to perform the differential expression analysis. The choice of ranking_method also specifies the statistical significance calculation.
    - default = information_coefficient
    - Pearson_correlation
    - signal_to_noise_ratio
    - t_test_statistic
    - Spearman correlation
- multi_phenotype_mode
  + Perform multiple differential expression runs for multiple phenotypes and produces the same outputs as a 2 phenotype run but with results stacked (with “genes_to_show” x 2 entries for each comparison). When using multiple phenotypes the outputs (heatmap or tables) have the same format but show a stack of results corresponding to each class comparison.
    - default = one_vs_all
    - all_pairs
- Outputs
  + Differential_expression_heatmap
    - A .png file of a heatmap showing the “n_top_hits_to_display” most differentially expressed genes in each phenotype comparison. Includes the confidence intervals for the metric and statistical significance results.
  + Differential_expression_results
    - A tab-separated TXT file which contains the results of the entire differential expression analysis, the confidence intervals for the metric and statistical significance results.
- Returns
  + This function returns (python) a data frame showing the same genes as the heatmap (gene_to_show x 2 entries).


## Use these files:
- http://datasets.genepattern.org/all_aml/all_aml_test.cls
- http://datasets.genepattern.org/all_aml/all_aml_test.gct

In [3]:
import genepattern
import cuzcatlan as cusca
import pandas as pd
import urllib.request
from cuzcatlan import compute_information_coefficient
from cuzcatlan import custom_pearson_corr
import validators
RANDOM_SEED = 20121020

#Note: definining here for simplicity, but this is a mirror of the function in cuzcatlan.
def differential_gene_expression(
        phenotypes:"CLS filename; input binary phenotype/class distinction",
        gene_expression:"GCT filename; data matrix with input gene expression profiles",
        output_filename:"Output files will have this name plus extensions .txt and .pdf",
        ranking_method:"The function to use to compute similarity between phenotypes and gene_expression",
        max_number_of_genes_to_show:"Maximum number of genes to show in the heatmap"=20,
        number_of_permutations:"Number of random permutations to estimate statistical significance (p-values and FDRs)"=10,
        title:"The title of the heatmap"=None,
        random_seed:"Random number generator seed (can be set to a user supplied integer for reproducibility)"=RANDOM_SEED):
    """
    Perform differential analysis on gene expression data of two phenotypes.
    """
    
#     :param output_filename: 
#     :param ranking_method:callable;
#     :param max_number_of_genes_to_show: int; 
#     :param number_of_permutations: int; 
#     :param title: str;
#     :param random_seed: int | array; f
#     :return: Dataframe; table of genes ranked by Information Coeff vs. phenotype

    data_df = pd.read_table(gene_expression, header=2, index_col=0)
    data_df.drop('Description', axis=1, inplace=True)
    
    if validators.url(phenotypes):
        urlfile, __ = urllib.request.urlretrieve(phenotypes)
    else:
        urlfile = phenotypes
        
    temp = open(urlfile)
    temp.readline()
    temp.readline()
    classes = [int(i) for i in temp.readline().strip('\n').split(' ')]
    classes = pd.Series(classes, index=data_df.columns)

    gene_scores = cusca.make_match_panel(
        features=data_df,
        target=classes,
        function=ranking_method,
        target_ascending=False,
        n_top_features=0.99,
        max_n_features=max_number_of_genes_to_show,
        n_samplings=30,
        n_permutations=number_of_permutations,
        random_seed=random_seed,
        target_type='binary',
        title=title,
        file_path_prefix=output_filename)

    return gene_scores

# genepattern.GPUIBuilder(cusca.differential_gene_expression, name="Differential Gene Expression! =D")
genepattern.GPUIBuilder(differential_gene_expression, name="Differential Gene Expression! =D")

GPUIBuilder(description='Perform differential analysis on gene expression data of two phenotypes.', function_import='differential_gene_expression', name='Differential Gene Expression! =D', params=[{'name': 'phenotypes', 'label': 'phenotypes', 'optional': False, 'default': '', 'description': 'CLS filename; input binary phenotype/class distinction', 'hide': False, 'choices': []}, {'name': 'gene_expression', 'label': 'gene_expression', 'optional': False, 'default': '', 'description': 'GCT filename; data matrix with input gene expression profiles', 'hide': False, 'choices': []}, {'name': 'output_filename', 'label': 'output_filename', 'optional': False, 'default': '', 'description': 'Output files will have this name plus extensions .txt and .pdf', 'hide': False, 'choices': []}, {'name': 'ranking_method', 'label': 'ranking_method', 'optional': False, 'default': '', 'description': 'The function to use to compute similarity between phenotypes and gene_expression', 'hide': False, 'choices': []}, {'name': 'max_number_of_genes_to_show', 'label': 'max_number_of_genes_to_show', 'optional': True, 'default': 20, 'description': 'Maximum number of genes to show in the heatmap', 'hide': False, 'choices': []}, {'name': 'number_of_permutations', 'label': 'number_of_permutations', 'optional': True, 'default': 10, 'description': 'Number of random permutations to estimate statistical significance (p-values and FDRs)', 'hide': False, 'choices': []}, {'name': 'title', 'label': 'title', 'optional': True, 'default': None, 'description': 'The title of the heatmap', 'hide': False, 'choices': []}, {'name': 'random_seed', 'label': 'random_seed', 'optional': True, 'default': 20121020, 'description': 'Random number generator seed (can be set to a user supplied integer for reproducibility)', 'hide': False, 'choices': []}])

In [4]:
de_results = differential_gene_expression(phenotypes="http://datasets.genepattern.org/all_aml/all_aml_test.cls", gene_expression="http://datasets.genepattern.org/all_aml/all_aml_test.gct", output_filename="Temp", ranking_method=custom_pearson_corr, max_number_of_genes_to_show=20, number_of_permutations=10, title="Temp_title", random_seed=20121020)

Index(['ALL 19769 TA+ Norel', 'ALL 406 TA+ (ML) Norel', 'ALL 4466 Norel',
       'ALL 1245 TA- Norel', 'ALL 16125 TA- Norel', 'ALL 23368 TA- Norel',
       'ALL R28 (ML) Relap', 'ALL 1234 ML', 'ALL 1305 ML', 'ALL 1421 ML',
       'ALL 1425 ML', 'ALL 1486 ML', 'ALL 1457 ML', 'ALL 1256 TA+ (ML) Norel',
       'ALL 1275 TA+ (ML) Norel', 'ALL 1101 TA- (ML) Norel',
       'ALL 1319 TA- (ML) Norel', 'ALL 522 TA- (ML) Norel', 'ALL SH 6',
       'ALL SH 12', 'ALL SH 15', 'AML 15 (PK) Norel', 'AML 19 (PK) Norel',
       'AML 10 (PK) Relap', 'AML 9 (PK) Relap', 'AML SH 5', 'AML SH 13',
       'AML SH 14', 'AML SH 16', 'AML SH 18', 'AML 17 ML', 'AML 25 ML',
       'AML 11 (ML)', 'AML 12 (ML)', 'AML 5 (ML) Relap'],
      dtype='object')
Index(['ALL 19769 TA+ Norel', 'ALL 406 TA+ (ML) Norel', 'ALL 4466 Norel',
       'ALL 1245 TA- Norel', 'ALL 16125 TA- Norel', 'ALL 23368 TA- Norel',
       'ALL R28 (ML) Relap', 'ALL 1234 ML', 'ALL 1305 ML', 'ALL 1421 ML',
       'ALL 1425 ML', 'ALL 1486 ML', 'ALL 

In [6]:
de_results = differential_gene_expression(phenotypes="http://datasets.genepattern.org/all_aml/all_aml_test.cls", gene_expression="http://datasets.genepattern.org/all_aml/all_aml_test.gct", output_filename="Temp", ranking_method=custom_pearson_corr, max_number_of_genes_to_show=20, number_of_permutations=10, title="Temp_title", random_seed=20121020)

Index(['ALL 19769 TA+ Norel', 'ALL 406 TA+ (ML) Norel', 'ALL 4466 Norel',
       'ALL 1245 TA- Norel', 'ALL 16125 TA- Norel', 'ALL 23368 TA- Norel',
       'ALL R28 (ML) Relap', 'ALL 1234 ML', 'ALL 1305 ML', 'ALL 1421 ML',
       'ALL 1425 ML', 'ALL 1486 ML', 'ALL 1457 ML', 'ALL 1256 TA+ (ML) Norel',
       'ALL 1275 TA+ (ML) Norel', 'ALL 1101 TA- (ML) Norel',
       'ALL 1319 TA- (ML) Norel', 'ALL 522 TA- (ML) Norel', 'ALL SH 6',
       'ALL SH 12', 'ALL SH 15', 'AML 15 (PK) Norel', 'AML 19 (PK) Norel',
       'AML 10 (PK) Relap', 'AML 9 (PK) Relap', 'AML SH 5', 'AML SH 13',
       'AML SH 14', 'AML SH 16', 'AML SH 18', 'AML 17 ML', 'AML 25 ML',
       'AML 11 (ML)', 'AML 12 (ML)', 'AML 5 (ML) Relap'],
      dtype='object')
Index(['ALL 19769 TA+ Norel', 'ALL 406 TA+ (ML) Norel', 'ALL 4466 Norel',
       'ALL 1245 TA- Norel', 'ALL 16125 TA- Norel', 'ALL 23368 TA- Norel',
       'ALL R28 (ML) Relap', 'ALL 1234 ML', 'ALL 1305 ML', 'ALL 1421 ML',
       'ALL 1425 ML', 'ALL 1486 ML', 'ALL 

In [7]:
de_results = differential_gene_expression(phenotypes="http://datasets.genepattern.org/all_aml/all_aml_test.cls", gene_expression="http://datasets.genepattern.org/all_aml/all_aml_test.gct", output_filename="Temp", ranking_method=custom_pearson_corr, max_number_of_genes_to_show=20, number_of_permutations=10, title="Temp_title", random_seed=20121020)

Index(['ALL 19769 TA+ Norel', 'ALL 406 TA+ (ML) Norel', 'ALL 4466 Norel',
       'ALL 1245 TA- Norel', 'ALL 16125 TA- Norel', 'ALL 23368 TA- Norel',
       'ALL R28 (ML) Relap', 'ALL 1234 ML', 'ALL 1305 ML', 'ALL 1421 ML',
       'ALL 1425 ML', 'ALL 1486 ML', 'ALL 1457 ML', 'ALL 1256 TA+ (ML) Norel',
       'ALL 1275 TA+ (ML) Norel', 'ALL 1101 TA- (ML) Norel',
       'ALL 1319 TA- (ML) Norel', 'ALL 522 TA- (ML) Norel', 'ALL SH 6',
       'ALL SH 12', 'ALL SH 15', 'AML 15 (PK) Norel', 'AML 19 (PK) Norel',
       'AML 10 (PK) Relap', 'AML 9 (PK) Relap', 'AML SH 5', 'AML SH 13',
       'AML SH 14', 'AML SH 16', 'AML SH 18', 'AML 17 ML', 'AML 25 ML',
       'AML 11 (ML)', 'AML 12 (ML)', 'AML 5 (ML) Relap'],
      dtype='object')
Index(['ALL 19769 TA+ Norel', 'ALL 406 TA+ (ML) Norel', 'ALL 4466 Norel',
       'ALL 1245 TA- Norel', 'ALL 16125 TA- Norel', 'ALL 23368 TA- Norel',
       'ALL R28 (ML) Relap', 'ALL 1234 ML', 'ALL 1305 ML', 'ALL 1421 ML',
       'ALL 1425 ML', 'ALL 1486 ML', 'ALL 

In [38]:
de_results = differential_gene_expression(phenotypes="http://datasets.genepattern.org/all_aml/all_aml_test.cls", gene_expression="http://datasets.genepattern.org/all_aml/all_aml_test.gct", output_filename="Temp", ranking_method=custom_pearson_corr, max_number_of_genes_to_show=20, number_of_permutations=10, title="Temp_title", random_seed=20121020)

Index(['ALL 19769 TA+ Norel', 'ALL 406 TA+ (ML) Norel', 'ALL 4466 Norel',
       'ALL 1245 TA- Norel', 'ALL 16125 TA- Norel', 'ALL 23368 TA- Norel',
       'ALL R28 (ML) Relap', 'ALL 1234 ML', 'ALL 1305 ML', 'ALL 1421 ML',
       'ALL 1425 ML', 'ALL 1486 ML', 'ALL 1457 ML', 'ALL 1256 TA+ (ML) Norel',
       'ALL 1275 TA+ (ML) Norel', 'ALL 1101 TA- (ML) Norel',
       'ALL 1319 TA- (ML) Norel', 'ALL 522 TA- (ML) Norel', 'ALL SH 6',
       'ALL SH 12', 'ALL SH 15', 'AML 15 (PK) Norel', 'AML 19 (PK) Norel',
       'AML 10 (PK) Relap', 'AML 9 (PK) Relap', 'AML SH 5', 'AML SH 13',
       'AML SH 14', 'AML SH 16', 'AML SH 18', 'AML 17 ML', 'AML 25 ML',
       'AML 11 (ML)', 'AML 12 (ML)', 'AML 5 (ML) Relap'],
      dtype='object')
Index(['ALL 19769 TA+ Norel', 'ALL 406 TA+ (ML) Norel', 'ALL 4466 Norel',
       'ALL 1245 TA- Norel', 'ALL 16125 TA- Norel', 'ALL 23368 TA- Norel',
       'ALL R28 (ML) Relap', 'ALL 1234 ML', 'ALL 1305 ML', 'ALL 1421 ML',
       'ALL 1425 ML', 'ALL 1486 ML', 'ALL 

In [5]:
de_results

,Score,0.95 MoE,p-value,FDR
Name,,,,
U46499_at,0.824544,NaN,0.000014,0.002381
X95735_at,0.775671,NaN,0.000014,0.002381
M63959_at,0.769661,NaN,0.000014,0.002381
L09209_s_at,0.747575,NaN,0.000014,0.002381
M84526_at,0.740053,NaN,0.000014,0.002381
X17042_at,0.731650,NaN,0.000014,0.002381
M14636_at,0.726834,NaN,0.000014,0.002381
M22960_at,0.726074,NaN,0.000014,0.002381
U59878_at,0.720990,NaN,0.000014,0.002381
